In [38]:
from __future__ import division
import sys
sys.path.append('./../cbp')
import utils
import cbp
import pandas as pd
import secrets
from cbp import Counties

In [2]:
# texas_api = Counties(state_fips='48', read_from='api', key=secrets.censuskey)
# texas_api.head()
# texas_api.to_csv('texas_cbp_2014.csv', index=False)

In [3]:
texas = Counties(state_fips='48', read_from='csv', filepath='texas_cbp_2014.csv')

In [4]:
texas.head()

,EMP,ESTAB,NAICS2012,NAICS2012_TTL,GEO_TTL,state,county
0,11738,940,00,Total for all sectors,Anderson County,48,001
1,5683,403,00,Total for all sectors,Andrews County,48,003
2,30525,1857,00,Total for all sectors,Angelina County,48,005
3,4131,503,00,Total for all sectors,Aransas County,48,007
4,1539,201,00,Total for all sectors,Archer County,48,009


In [5]:
len(texas)

144585

In [6]:
fips = []
for county in "Bastrop, Caldwell, Hays, Travis, Williamson".split(", "):
    fips.append(texas[(texas.NAICS2012 == '00') & (texas.GEO_TTL.str.contains(county))]['county'].iloc[0])
print fips

['021', '055', '209', '453', '491']


In [7]:
austin_region = texas.two_digit(county=fips)

In [8]:
austin_region.head()

,EMP,ESTAB,NAICS2012,NAICS2012_TTL,GEO_TTL,state,county
10,12475,1175,00,Total for all sectors,Bastrop County,48,021
27,6116,580,00,Total for all sectors,Caldwell County,48,055
104,46675,3660,00,Total for all sectors,Hays County,48,209
225,544038,32217,00,Total for all sectors,Travis County,48,453
244,136393,9252,00,Total for all sectors,Williamson County,48,491


In [9]:
aggs = {
    'EMP':'sum',
    'ESTAB':'sum',
    'NAICS2012_TTL':'first'
}
austin_msa = austin_region.groupby('NAICS2012',as_index=False).agg(aggs)
austin_msa['GEO_TTL'] = 'Austin-Round Rock MSA'

In [10]:
austin_msa.head()

,NAICS2012,NAICS2012_TTL,ESTAB,EMP,GEO_TTL
0,00,Total for all sectors,46884,745697,Austin-Round Rock MSA
1,11,"Agriculture, Forestry, Fishing and Hunting",28,21,Austin-Round Rock MSA
2,21,"Mining, Quarrying, and Oil and Gas Extraction",216,2440,Austin-Round Rock MSA
3,22,Utilities,116,2282,Austin-Round Rock MSA
4,23,Construction,3665,45334,Austin-Round Rock MSA


In [11]:
texas_total_emp = texas.two_digit().groupby('NAICS2012').agg('sum').EMP
austin_msa_emp = austin_msa.set_index('NAICS2012').EMP
lq = pd.DataFrame(utils.location_quotient(austin_msa_emp, texas_total_emp))
austin_msa = austin_msa.merge(lq, left_on='NAICS2012', right_index=True)

In [12]:
austin_msa

,NAICS2012,NAICS2012_TTL,ESTAB,EMP,GEO_TTL,location_quotient
0,00,Total for all sectors,46884,745697,Austin-Round Rock MSA,1.000000
1,11,"Agriculture, Forestry, Fishing and Hunting",28,21,Austin-Round Rock MSA,0.069275
2,21,"Mining, Quarrying, and Oil and Gas Extraction",216,2440,Austin-Round Rock MSA,0.160404
3,22,Utilities,116,2282,Austin-Round Rock MSA,0.816960
4,23,Construction,3665,45334,Austin-Round Rock MSA,1.003203
5,31-33,Manufacturing,1299,42592,Austin-Round Rock MSA,0.709544
6,42,Wholesale Trade,2032,34186,Austin-Round Rock MSA,0.886169
7,44-45,Retail Trade,5886,98831,Austin-Round Rock MSA,1.026170
8,48-49,Transportation and Warehousing,790,13551,Austin-Round Rock MSA,0.441452
9,51,Information,1129,31857,Austin-Round Rock MSA,1.831615


In [13]:
texas06 = Counties(state_fips='48', year=2006, read_from='csv', filepath='texas_cbp_2006.csv')

In [14]:
austin_region_06 = texas06.two_digit(county=fips)

In [15]:
aggs = {
    'EMP':'sum',
    'ESTAB':'sum'
}
austin_msa_06 = austin_region_06.groupby('NAICS2012',as_index=False).agg(aggs)
austin_msa_06['GEO_TTL'] = 'Austin-Round Rock MSA'

In [16]:
austin_msa_emp_06 = austin_msa_06.set_index('NAICS2012').EMP
texas_total_emp_06 = texas06.two_digit().groupby('NAICS2012').agg('sum').EMP

In [19]:
ss= utils.shift_share(small_old=austin_msa_emp_06,
                  small_new=austin_msa_emp,
                  large_old=texas_total_emp_06,
                  large_new=texas_total_emp)

In [21]:
ss.head()

,small_old,small_new,large_old,large_new,large_growth_share,large_industry_growth_rate,industry_mix,small_industry_growth_rate,local_competitiveness
NAICS2012,,,,,,,,,
00,609646,745697,8711361,9598578,62089.987452,0.101846,0.000000,0.223164,73961.012548
11,55,21,5343,3902,5.601528,-0.269699,-20.434955,-0.618182,-19.166573
21,1439,2440,141126,195803,146.556349,0.387434,410.961068,0.695622,443.482583
22,789,2282,28784,35955,80.356469,0.249131,116.208254,1.892269,1296.435277
23,41128,45334,551643,581674,4188.721002,0.054439,-1949.745853,0.102266,1967.024851


In [39]:
res = pd.DataFrame()

In [40]:
res.loc['small_growth', 'description'] = 'Growth in smaller geography'
res.loc['small_growth', 'absolute'] = ss_no00.small_new.sum() - ss_no00.small_old.sum()
res.loc['small_growth', 'percentage'] =( ss_no00.small_new.sum() - ss_no00.small_old.sum())/ss_no00.small_old.sum()

In [42]:
res.loc['large_growth', 'description'] = 'Growth in larger geography'
res.loc['large_growth', 'absolute'] = ss_no00.large_new.sum() - ss_no00.large_old.sum()
res.loc['large_growth', 'percentage'] =( ss_no00.large_new.sum() - ss_no00.large_old.sum())/ss_no00.large_old.sum()

In [44]:
res.loc['large_growth_share', 'description'] = 'Growth attributable to larger geography growth rate'
res.loc['large_growth_share', 'absolute'] = ss_no00.large_growth_share.sum()
res.loc['large_growth_share', 'percentage'] = ss_no00.large_growth_share.sum() / res.loc['small_growth', 'absolute']

In [46]:
res.loc['industry_mix', 'description'] = 'Growth attributable to industry mix'
res.loc['industry_mix', 'absolute'] = ss_no00.industry_mix.sum()
res.loc['industry_mix', 'percentage'] = ss_no00.industry_mix.sum() / res.loc['small_growth', 'absolute']

In [48]:
res.loc['local_competitiveness', 'description'] = 'Growth attributable to local competitiveness'
res.loc['local_competitiveness', 'absolute'] = ss_no00.local_competitiveness.sum()
res.loc['local_competitiveness', 'percentage'] = ss_no00.local_competitiveness.sum() / res.loc['small_growth', 'absolute']

In [49]:
res

,description,absolute,percentage
small_growth,Job growth in smaller geography,136761.000000,0.227547
large_growth,Job growth in larger geography,784017.000000,0.090705
large_growth_share,Growth attributable to larger geography growth...,61211.667818,0.447581
industry_mix,Growth attributable to industry mix,-7224.713613,-0.052827
local_competitiveness,Growth attributable to local competitiveness,82774.045795,0.605246


In [51]:
res.loc[['large_growth_share', 'industry_mix', 'local_competitiveness'], 'absolute'].sum()

136761.0

In [23]:
ss_no00 = ss[ss.index != '00']

In [26]:
ss_no00.local_competitiveness.sum()

82774.0457949365

In [ ]:
df.columns = ['small']

In [ ]:
df

In [ ]:
larger = texas.two_digit().groupby('NAICS2012').agg('sum').EMP

In [ ]:
df['large'] = larger

In [ ]:
df

In [ ]:
total_emp_row = df.loc['00']

In [ ]:
total_emp_row.small

In [ ]:
all(larger.index == small.index)